In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip3 install --user Flask

In [11]:
!pip install flask-ngrok

from flask import Flask
from flask import request
from flask_ngrok import run_with_ngrok
import flask

# import numpy, panda, keras and tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import models, layers
from tensorflow.keras.models import Sequential, load_model
import os


In [12]:
!pip install h5py

In [13]:
os.chdir("/content/drive/My Drive/Colab Notebooks")
!ls

Flask_Dep.ipynb  Flask_Depl.ipynb  model.h5  static  templates


In [14]:
# !export FLASK_ENV=development

In [15]:
import re
import base64

from PIL import Image
from io import BytesIO

def base64_to_pil(img_base64):
    """
    Convert base64 image data to PIL image
    """
    image_data = re.sub('^data:image/.+;base64,', '', img_base64)
    pil_image = Image.open(BytesIO(base64.b64decode(image_data)))
    return pil_image


def np_to_base64(img_np):
    """
    Convert numpy image (RGB) to base64 string
    """
    img = Image.fromarray(img_np.astype('uint8'), 'RGB')
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    return u"data:image/png;base64," + base64.b64encode(buffered.getvalue()).decode("ascii")

In [16]:
import os
import sys

# Flask
from flask import Flask, redirect, url_for, request, render_template, Response, jsonify, redirect

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Declare a flask app
app = Flask(__name__, template_folder=os.path.join('/content/drive/My Drive/Colab Notebooks/templates'), static_folder=os.path.join('/content/drive/My Drive/Colab Notebooks/static'))
run_with_ngrok(app)  # Start ngrok when app is run

# Loading my already trained model
# MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/model.h5'
model = load_model(os.path.join('/content/drive/My Drive/Colab Notebooks/model.h5'))

print('Model loaded. Start serving...')


def model_predict(img, model):
    img = img.resize((224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # The input must be preprocessed the same way as the training image
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x)

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        # Get the image from post request
        img = base64_to_pil(request.json)

        # Make prediction
        preds = model_predict(img, model)

        if(preds[0][0] >= 0.5):
            prediction = 'No Gun detected'
        else:
          prediction = 'Gun detected'

        result = prediction.replace('_', ' ').capitalize()
        
        # Serialize the result, you can add additional fields
        return jsonify(result=result)

    return None


if __name__ == '__main__':
    app.run()



Model loaded. Start serving...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://69f217b27326.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Jun/2021 04:12:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:12:57] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:12:57] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:12:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Jun/2021 04:13:11] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:13:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:13:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:34:51] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:44:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 04:44:49] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2021 05:12:28] "POST /predict HTTP/1.1" 200 -
